# setup

In [43]:
import pandas as pd
import tqdm
import os
import numpy as np

In [2]:
input_file = 'RECORDS.txt'
root_dir = 'Training_WFDB'

In [3]:
files = pd.read_csv(input_file, header=None).values.flatten()
folder = root_dir
n = len(files)
files

array(['A0001', 'A0002', 'A0003', ..., 'A6875', 'A6876', 'A6877'],
      dtype=object)

# draft

In [4]:
for i, f in enumerate(tqdm.tqdm(files)):
    break
i, f

  0%|          | 0/6877 [00:00<?, ?it/s]


(0, 'A0001')

In [29]:
with open(os.path.join(root_dir, f) + '.hea') as header:
    text = header.read()
    print(text[text.find('#Dx'):].split('#')[1])

Dx: 59118001



# loop

In [51]:
def get_row(exam_id, text):
    return [exam_id] + text[text.find('#Dx'):].split('#')[1][4:-1].split(',')

In [66]:
def label2onehot(exam_id, text):
    # ['exam_id' [0], 'AF' [1], 'I-AVB' [2], 'LBBB' [3], 'RBBB' [4], 'PAC' [5], 'PVC' [6], 'STD' [7], 'STE' [8]]
    onehot = [np.nan, False, False, False, False, False, False, False, False]
    onehot[0] = exam_id

    labels = text[text.find('#Dx'):].split('#')[1][4:-1].split(',')
    match = False
    for label in labels:
        if label == '164884008':
            onehot[6] = True
            match = True
        if label == '164889003':
            onehot[1] = True
            match = True
        if label == '164909002':
            onehot[3] = True
            match = True
        if label == '164931005':
            onehot[8] = True
            match = True
        if label == '270492004':
            onehot[2] = True
            match = True
        if label == '284470004':
            onehot[5] = True
            match = True
        if label == '426783006': # normal
            match = True
        if label == '429622005':
            onehot[7] = True
            match = True
        if label == '59118001':
            onehot[4] = True
            match = True
    assert match != False
    return onehot

In [67]:
data = []
for f in tqdm.tqdm(files):
    exam_id = int(f[1:])
    with open(os.path.join(root_dir, f) + '.hea') as header:
        text = header.read()
        # data.append(text[text.find('#Dx'):].split('#')[1])
        # data.append(text[text.find('#Dx'):].split('#')[1][4:-1].split(','))
        # data.append(get_row(exam_id, text))
        data.append(label2onehot(exam_id, text))

100%|██████████| 6877/6877 [00:00<00:00, 45261.82it/s]


In [68]:
data

[[1, False, False, False, True, False, False, False, False],
 [2, False, False, False, False, False, False, False, False],
 [3, True, False, False, False, False, False, False, False],
 [4, True, False, False, False, False, False, False, False],
 [5, False, False, False, False, False, True, False, False],
 [6, False, False, False, True, False, False, False, False],
 [7, True, False, False, False, False, False, False, False],
 [8, False, False, False, False, False, False, True, False],
 [9, True, False, False, False, False, False, False, False],
 [10, False, False, False, True, False, False, False, False],
 [11, False, False, True, False, False, False, False, False],
 [12, False, False, False, False, False, True, False, False],
 [13, False, False, False, False, False, False, True, False],
 [14, False, False, False, False, False, False, True, False],
 [15, False, False, False, True, False, False, False, False],
 [16, False, False, False, False, False, False, False, False],
 [17, True, Fal

In [61]:
# columns = ['exam_id', 'First_label', 'Second_label', 'Third_label']

# df = pd.DataFrame(data, columns = columns)
# df

,exam_id,First_label,Second_label,Third_label
0,1,59118001,None,None
1,2,426783006,None,None
2,3,164889003,None,None
3,4,164889003,None,None
4,5,164884008,None,None
...,...,...,...,...
6872,6873,270492004,None,None
6873,6874,429622005,None,None
6874,6875,164909002,None,None
6875,6876,164889003,None,None


In [69]:
columns = ['exam_id', 'AF', 'I-AVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']
dx = pd.DataFrame(data, columns = columns)
dx

,exam_id,AF,I-AVB,LBBB,RBBB,PAC,PVC,STD,STE
0,1,False,False,False,True,False,False,False,False
1,2,False,False,False,False,False,False,False,False
2,3,True,False,False,False,False,False,False,False
3,4,True,False,False,False,False,False,False,False
4,5,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...
6872,6873,False,True,False,False,False,False,False,False
6873,6874,False,False,False,False,False,False,True,False
6874,6875,False,False,True,False,False,False,False,False
6875,6876,True,False,False,False,False,False,False,False


In [70]:
dx.to_csv('dx.csv', index = None)